### Librerías

In [1]:
import numpy as np
import pandas as pd
import random

### Cronograma de actores y tomas

In [10]:
# Función para obtener un cronograma de actores y tomas con participación aleatoria

def get_schedule(act, tom):
    
    data_set = np.zeros(shape=(tom, act))
    
    for i in range(tom):
        for j in range(act):
            data_set[i][j] = random.randint(0,1)
            
    return data_set

In [11]:
# Cantidad de actores y número de tomas a generarse
num_act = 12
num_tom = 28

data_set = get_schedule(num_act, num_tom)

In [13]:
rows, columns = data_set.shape
print(f'Tomas: {rows} y Actores: {columns}')

Tomas: 28 y Actores: 12


### Funciones para el desarrollo del algoritmo

In [14]:
# Función para definir si existen tomas en las que se repiten actores

def val_rep(list1, list2):
    '''
    m_list -> lista mayor
    n_list -> lista menor
    '''
    if len(list1) >= len(list2):
        m_list = list1
        n_list = list2
    else:
        m_list = list2
        n_list = list1
    
    j = 0
    
    for item in n_list:
        if item in m_list:
            j += 1
        else:
            pass
    
    if j > 0:
        return False
    else:
        return True

In [15]:
# Función que obtine los actores que aparecen en la toma

def act_tom(idx_i, data_set):
    
    '''
    idx_i: indice de la toma (int)
    data_set: cronograma tomas/actores (2D array)
    tomas: número de tomas (list)
    '''
    
    # Actores que aprecen en la toma
    data_ex = mod_dataSet[idx_i, :]

    # Indice de los actores que aprecen en la toma n
    loct_n = np.argwhere(data_ex == 1)
    loct_n  = loct_n.flatten()

    return loct_n

In [16]:
# Función para buscar las tomas que pueden compaginar con una toma seleccionada

def sup_list(list_2, dataSet, loct_n):
    for item in list_2:
        
        loct_m = act_tom(item, dataSet)

        if val_rep(loct_n, loct_m):
            return item
        else:
            pass

In [17]:
# Función para mover las tomas de tal manera que reubico tomas donde no se repiten los actores

def reub_tomas(list_tomas, mod_dataSet, tomas, list_t):
    
    #mod_dataSet = np.array(data_set, copy = True)
    rows, columns = data_set.shape
    
    for item in list_tomas:
    
        act_n = item
        loct_n = act_tom(act_n, mod_dataSet)

        tom_res = tomas.copy()
        tom_res.remove(act_n)

        sum_act = mod_dataSet[act_n].sum()

        list_j = []

        for item in tom_res:
            if mod_dataSet[item].sum() <= columns - sum_act:
                list_j.append(item)
            else:
                pass

        idx_2 = sup_list(list_j, mod_dataSet, loct_n)
        loct_m = np.argwhere(mod_dataSet[idx_2, :] == 1)
        if loct_n.size != 0 and loct_m.size != 0:
            loct_m  = loct_m.flatten()
            mod_dataSet[act_n, loct_m] = 1
            mod_dataSet[idx_2, loct_m] = 0
            list_t[act_n].append(idx_2)
        else:
            pass

    return mod_dataSet, list_t

In [18]:
# Función para agrupar las tomas en conjuntos de 6

def op_schedule(rows, mod_dataSet, list_t, n):
    
    idx_i = []
    
    for i in range(rows):
        if mod_dataSet[i].sum() == 0:
            idx_i.append(i)
        else:
            pass
    
    mod_dataSet = np.delete(mod_dataSet, idx_i, 0)

    for item in idx_i:
        list_t.remove(list_t[item])
    
    # Combinar tomas de tal manera que haya conjuntos de n
    new_mod = np.array(mod_dataSet, copy = True)
    n_rows, n_columns = new_mod.shape
    conj_cnd = n
    sub_c = n_rows // conj_cnd
    last_sub = n_rows - sub_c*conj_cnd
    if last_sub > 0:
        sub_c += 1
    else:
        pass
    
    schedule = np.zeros((sub_c, n_columns))
    j = 0
    tomas_seg = []

    for i in range(sub_c):
        schedule[i] = np.sum(new_mod[j:j+n, :], axis=0)
        j += n
    
    return schedule, len(list_t)

### Algoritmo hecho en un sólo bucle y con las tomas en orden

In [19]:
rows, columns = data_set.shape
tomas = [j for j in range(rows)]
list_tomas = [i for i in range(rows)]
list_t = []
for i in range(1, rows + 1):
    item = []
    item.append(i)
    list_t.append(item)
mod_dataSet = np.array(data_set, copy = True)

In [20]:
new_dataset, tomas_mov = reub_tomas(list_tomas, mod_dataSet, tomas, list_t)
print(new_dataset)
print(tomas_mov)

[[0. 0. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1.]
 [0. 1. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1.]
 [0. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1.]
 [0. 1. 1. 0. 1. 0. 0. 1. 0. 0. 1. 1.]
 [1. 0. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1.]
 [1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 1. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 1. 1. 0. 0. 1. 1. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 0. 0. 1. 1. 1. 0. 1. 0. 0. 1.]
 [1. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 1. 0. 1. 1. 0. 1. 0.]
 [0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1.]
 [0. 1. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0.]
 [1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 0.]
 [0. 0. 1. 0. 0. 0. 1. 1. 1. 0. 1. 1.]
 [1. 0. 0. 0. 1. 1. 1. 0. 1. 0. 1. 1.]
 [0. 0. 0. 1. 0. 1. 0. 1. 0. 1. 1. 0.]
 [1. 1. 0. 1. 1. 1. 0. 0. 1. 1. 1. 0.]
 [0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0.]
 [0. 1. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0.]
 [1. 1. 0. 1. 0. 0. 0. 0. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 1. 1. 0.

Ingresar el número máximo de tomas por día que se pueden realizar

In [21]:
sub_conj = 7
schedule, tomas_per = op_schedule(rows, new_dataset, tomas_mov, sub_conj)
print(f"El número de tomas se redujo a {tomas_per} tomas")
print(schedule)

El número de tomas se redujo a 25 tomas
[[3. 5. 7. 4. 7. 3. 4. 3. 4. 2. 3. 6.]
 [3. 3. 4. 4. 2. 4. 4. 6. 4. 2. 5. 3.]
 [3. 4. 3. 4. 3. 4. 3. 3. 4. 5. 6. 3.]
 [3. 3. 0. 1. 0. 3. 3. 3. 1. 1. 2. 3.]]
